## Własne warstwy w Kerasie

Jenda z kluczowych funkcjonalności bibliotek do tworzenia modeli uczenia maszynowego. Keras również oferuje możliwość tworzenia własnych warstw.

### UWAGA!
W przypadku korzystania z własnych modeli w TensorFlow.js istotne jest, aby warstwy były pisane w sposób zgodny z API Kerasa. W warstwie należy definować metody `build`, `call` oraz `get_config`.

### Warstwa typu Residual

In [ ]:
from keras import layers, models


class ResidualBlock(layers.Layer):  # dziedziczymy po klasie Layer z modułu keras.layers
    def __init__(self, filters: int, **kwargs):
        # wywołujemy konstruktor klasy bazowej
        # pozwala to na przekazanie dodatkowych argumentów do konstruktora klasy bazowej
        # np. `name` lub `trainable`
        super(ResidualBlock, self).__init__(**kwargs)

        # Nie tworzymy warstw w __init__
        # Warstwy tworzymy w metodzie `build`, która jest wywoływana automatycznie
        self.filters = filters

    def get_config(self):
        # Aby porządnie zdefiniować tę metodę zalecam pobrać konfigurację z klasy bazowej
        super_config = super(ResidualBlock, self).get_config()
        # Dodajemy do niej nasze własne pola
        super_config.update({"filters": self.filters})
        return super_config

    def build(self, input_shape):
        # Warstwa build daje nam dostęp do rozmiaru wejścia w parametrze `input_shape`
        # W Kerasie nie trzeba podawać rozmiaru wejścia! Keras sam sobie to obliczy

        # blok splotu
        self.conv = layers.Conv2D(self.filters, kernel_size=(3, 3), padding="same")
        self.bn = layers.BatchNormalization()
        self.activation = layers.Activation("relu")

        # zamiast self.add możemy po prostu użyć operatora `+`
        self.add = layers.Add()

        # parametr skalujący do pomnożenia `inputs` przed dodaniem
        self.scale = self.add_weight((1, 1, input_shape[-1]), initializer="ones")

    def call(self, inputs):
        # Warstwy są wywoływane w metodzie `call`
        # Tutaj definiujemy przepływ danych przez warstwę

        x = self.conv(inputs)
        x = self.bn(x)
        x = self.activation(x)

        # Dodajemy wejście do wyjścia
        return self.add([inputs * self.scale, x])


model = models.Sequential(
    [
        layers.Input((28, 28, 1)),
        layers.Conv2D(32, kernel_size=(3, 3), padding="same"),
        ResidualBlock(32, name="rs_block"),
        ResidualBlock(32),
        layers.GlobalAveragePooling2D(),
        layers.Dense(10, activation="softmax"),
    ]
)

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 28, 28, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rs_block (ResidualBlock)        │ (None, 28, 28, 32)     │         9,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_3                │ (None, 28, 28, 32)     │         9,408 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 32)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           330 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,466 (76.04 KB)

 Trainable params: 19,338 (75.54 KB)

 Non-trainable params: 128 (512.00 B)

### Warswa z kilkoma wejściami

In [7]:
from keras import layers, models, ops


class MultiInputLayer(layers.Layer):
    def __init__(self, filters: int, **kwargs):
        super(MultiInputLayer, self).__init__(**kwargs)

        # Nie tworzymy warstw w __init__
        # Warstwy tworzymy w metodzie `build`, która jest wywoływana automatycznie
        self.filters = filters

    def get_config(self):
        super_config = super(ResidualBlock, self).get_config()
        super_config.update({"filters": self.filters})
        return super_config

    def build(self, input_shape):
        # (B, H, W, C), (B, N)
        input_shape_a, input_shape_b = input_shape

        self.block = models.Sequential(
            [
                layers.Conv2D(self.filters, kernel_size=(3, 3), padding="same"),
                layers.BatchNormalization(),
                layers.Activation("relu"),
            ]
        )

        self.linear = models.Sequential(
            [
                layers.Dense(self.filters, activation="relu"),
                layers.Reshape((1, 1, self.filters)),
            ]
        )

    def call(self, inputs):
        inputs_a, inputs_b = inputs

        x = self.block(inputs_a)
        y = self.linear(inputs_b)

        return x + y


inputs = layers.Input((28, 28, 1))
aux_inputs = layers.Input((10,))

x = MultiInputLayer(32)([inputs, aux_inputs])
outputs = layers.Conv2D(3, 3, padding="same")(x)

model = models.Model([inputs, aux_inputs], outputs)

model.summary()

images = ops.ones((32, 28, 28, 1))
aux = ops.ones((32, 10))

model([images, aux]).shape


Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_14      │ (None, 28, 28, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_15      │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_input_layer_3 │ (None, 28, 28,    │        800 │ input_layer_14[0… │
│ (MultiInputLayer)   │ 32)               │            │ input_layer_15[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_11 (Conv2D)  │ (None, 28, 28, 3) │        867 │ multi_input_laye… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,667 (6.51 KB)

 Trainable params: 1,603 (6.26 KB)

 Non-trainable params: 64 (256.00 B)

TensorShape([32, 28, 28, 3])